In [180]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

## Set up tools

In [181]:
# Define which tools the agent can use to answer user queries
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="Useful for when you need to answer questions about current events. You should ask targeted queries."
    )
]

## Set up prompt template (we can tune it)

In [190]:
# Set up the base template
template = """
Your decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications. If you have completed all your tasks, make sure to use the "finish" command.

Goal:
{input}

You have access to the following tools:
{tools}

You must output using the following format:

Thought: you should always think about what to do
Reasoning: reasoning
Plan: - short bulleted\n- list that conveys\n- next-step plan
Relection: constructive self-reflection
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Reasoning/Plan/Relection/Action/Action Input/Observation can repeat N times)
When you want to finish:
Final Answer: the final answer to achieve the original goal. 

Begin! 

{agent_scratchpad}

Determine which next command to use, and respond using the format specified above:
"""

In [207]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for n_round, (action, observation) in enumerate(intermediate_steps):
#             thoughts += f"Round {n_round+1}\n"
            thoughts += f"{action.log}"
            thoughts += f"\nObservation: {observation}\n\n"
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        final_prompt = self.template.format(**kwargs)
        print("\n\033[91m" + "Final Prompt to OpenAI" + "\033[0m")
        print("\033[94m" + f"{final_prompt}" + "\033[0m")
        input_prompt = final_prompt
        return final_prompt

In [199]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

## Output Parser

In [208]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        output_text = llm_output
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [209]:
output_parser = CustomOutputParser()

## Set up a LM

In [210]:
llm = ChatOpenAI(temperature=0)

## Set up the Agent

In [211]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [212]:
agent_executor.run("Find the upcoming holiday and generate a home recipe to celebrate.")
# agent_executor.run("Obtain the details for the latest match in the UEFA Champions League, including the team scores, goal scorers, and the times at which the goals were scored.")



> Entering new AgentExecutor chain...
Final Prompt to OpenAI

Your decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications. If you have completed all your tasks, make sure to use the "finish" command.

Goal:
Find the upcoming holiday and generate a home recipe to celebrate.

You have access to the following tools:
Search: Useful for when you need to answer questions about current events. You should ask targeted queries.

You must output using the following format:

Thought: you should always think about what to do
Reasoning: reasoning
Plan: - short bulleted
- list that conveys
- next-step plan
Relection: constructive self-reflection
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Reasoning/Plan/Relection/Action/Action Input/Observation can repeat N times)
When you want to fini

Thought: I have decided to make a vegetarian version of a traditional Southern dish for Confederate Memorial Day.
Reasoning: This will allow me to celebrate the holiday in a way that aligns with my dietary restrictions and personal values.
Plan:
- Choose a vegetarian recipe for a traditional Southern dish, such as vegetarian jambalaya or black-eyed pea salad.
- Gather all necessary ingredients and begin cooking.
- Serve the dish with a side of cornbread and sweet tea.
Reflection: I need to make sure that the recipe I choose is still respectful of the holiday's history and significance, even if it is a vegetarian version.
Action: Search
Action Input: "Vegetarian Southern recipes for Confederate Memorial Day"

Observation:20 Vegetarian Memorial Day Recipes You'll Want to Make All Summer · Ultimate Veggie Burgers · Charred Avocado Salad with Cilantro Mayo · Nina's ...
Final Prompt to OpenAI

Your decisions must always be made independently without seeking user assistance. Play to your str

"For Confederate Memorial Day, I will make a vegetarian version of a traditional Southern dish, such as vegetarian jambalaya or black-eyed pea salad, and serve it with cornbread and sweet tea. I will make sure that the recipe I choose is respectful of the holiday's history and significance."

In [222]:
agent_executor.run("Get weather forcast for Pittsburgh tomorrow.")



> Entering new AgentExecutor chain...
Final Prompt to OpenAI

Your decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications. If you have completed all your tasks, make sure to use the "finish" command.

Goal:
Get weather forcast for Pittsburgh tomorrow.

You have access to the following tools:
Search: Useful for when you need to answer questions about current events. You should ask targeted queries.

You must output using the following format:

Thought: you should always think about what to do
Reasoning: reasoning
Plan: - short bulleted
- list that conveys
- next-step plan
Relection: constructive self-reflection
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Reasoning/Plan/Relection/Action/Action Input/Observation can repeat N times)
When you want to finish:
Final Answer: the 

'The weather forecast for Pittsburgh tomorrow is a high of 53°F and a low of 34°F with a 24% chance of precipitation and west winds at 11 mph.'